### dependencies

In [83]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import pickle
import tensorflow
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, Activation
import gc
import warnings
warnings.filterwarnings('ignore')

### creating the dataset

In [4]:
path = '../input/amazonsatelliteimages/train-jpg/train-jpg'

for img in os.listdir(path):
    img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
    plt.imshow(img_arr)
    # printing one image
    break

In [5]:
gc.collect()
IMG_SIZE = 50

train_data = []

def create_train_data():

    for img in os.listdir(path):
      try:
        img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
        new_img_arr = cv2.resize(img_arr, (IMG_SIZE, IMG_SIZE))
        train_data.append(new_img_arr)
      except Exception as e:
        print(e)
        
create_train_data()

In [36]:
labels = pd.read_csv('../input/planet-understanding-the-amazon-from-space/train_v2.csv/train_v2.csv')

X = np.array(train_data)
y = np.array(labels['tags'])

y.shape

### encoding output variable

In [44]:
encode = LabelEncoder().fit(y)
encoded_y = encode.transform(y)
y = np_utils.to_categorical(encoded_y)

pickle.dump(X, open('./X.pkl','wb'))
pickle.dump(y, open('./y.pkl','wb'))

In [45]:
pd.DataFrame(y).describe()

### creating the model

In [100]:
train_X = pickle.load(open('./X.pkl', 'rb'))
train_y = pickle.load(open('./y.pkl', 'rb'))

# normalizing features
train_X = train_X/255.0

n_epochs = 50
b_size = 1000

model = Sequential()
# ADDING LAYERS

# input layer
model.add(Conv2D(164, (3, 3), input_shape=train_X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(164, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))

# output neuron
model.add(Flatten())
model.add(Dense(449))
model.add(Activation('softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [101]:
gc.collect()
model.summary()

### training the model

In [107]:
gc.collect()

In [108]:
model.fit(train_X, train_y, epochs=n_epochs, batch_size=b_size, validation_split=0.2)